# Homework 3 - MapReduce

In this homework, we are practicing the MapReduce programming paradigm. 

You are required to turn in this notebook as BDM\_HW3\_MR\_**NetId**.ipynb. You will be asked to complete each task using the accompanied *mapreduce* package (as **mapreduce.py**) and/or MRJob using one or more MapReduce "steps". For each such step (with **mr.run()**), you are expected to supply a mapper and a reducer as needed. Or if you're using MRJob, please call **mr.runJob()** instead. Below are sample usage of the package:

```python
    # Run on input1 using your mapper1 and reducer1 function
    output = list(mr.run(input1, mapper1, reducer1))

    # Run on input2 using only your mapper2, no reduce phase
    output = list(mr.run(enumerate(input2), mapper2, combiner2))
    
    # Run on input3 using 2 nested MapReduce jobs
    output = mr.run(mr.run(input3, mapper3, reducer3), mapper4)
```
    
Please note that the input must be an iteratable of **key/value pairs**.

If your inpu tdata does not have a key, you can simply add a null or index key through **enumerator(input)**. 

The output of the mr.run() is always a **generator**. You have to cast it to a list if you'd like to view, index or print it out.

## Task (10 points)

There is only one task in this homework. You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [112]:
import csv
import mapreduce as mr

In [113]:
def mapper1(_, row):
    for i in row['To'].split(';'):
        if i.split('@')[1] == 'enron.com' and row['From'].split('@')[1] == 'enron.com':
            for j in range(len(row['From'].split('@')[0].split('.'))):
                if len(row['From'].split('@')[0].split('.')) == 1:
                    from_name = row['From'].split('@')[0].split('.').capitalize()
                elif len(row['From'].split('@')[0].split('.')) == 2:
                    from_name = row['From'].split('@')[0].split('.')[0].capitalize() +" "+  row['From'].split('@')[0].split('.')[1].capitalize()
                else:
                    from_name = row['From'].split('@')[0].split('.')[0].capitalize() +" "+  row['From'].split('@')[0].split('.')[1].capitalize() + row['From'].split('@')[0].split('.')[2].capitalize()
            for k in range(len(i.split('@')[0].split('.'))):
                if len(i.split('@')[0].split('.')) == 1:
                    to_name = i.split('@')[0].split('.')[0].capitalize()
                elif len(i.split('@')[0].split('.')) == 2:
                    to_name = i.split('@')[0].split('.')[0].capitalize() +" "+ i.split('@')[0].split('.')[1].capitalize()
                else:
                    to_name = i.split('@')[0].split('.')[0].capitalize() +" "+ i.split('@')[0].split('.')[1].capitalize()+" "+ i.split('@')[0].split('.')[2].capitalize()
            yield(from_name, to_name)

In [114]:
def reducer1(from_name, to_list):
    yield(from_name, set(to_list))

In [116]:
def mapper2(from_name, to_list):
    for i in range(len(list(to_list))):
        yield(sorted((list(to_list)[i], from_name)), 1)

In [117]:
def reducer2(from_to, counts):
    yield(from_to, sum(counts))

In [126]:
def mapper3(from_to, counts):
    if counts == 2:
        yield('recipropcal', from_to[0] +' : '+ from_to[1])

In [127]:
with open('enron_mails_small.csv', 'r') as fi:
    reader = enumerate(csv.DictReader(fi))
    output = list(mr.run(mr.run(mr.run(reader, mapper1, reducer1), mapper2, reducer2), mapper3))
print(len(output))
output

35


[('recipropcal', 'Brenda Whitehead : Elizabeth Sager'),
 ('recipropcal', 'Carol Clair : Debra Perlingiere'),
 ('recipropcal', 'Carol Clair : Mark Taylor'),
 ('recipropcal', 'Carol Clair : Richard Sanders'),
 ('recipropcal', 'Carol Clair : Sara Shackleton'),
 ('recipropcal', 'Carol Clair : Tana Jones'),
 ('recipropcal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('recipropcal', 'Drew Fossum : Susan Scott'),
 ('recipropcal', 'Elizabeth Sager : Janette Elbertson'),
 ('recipropcal', 'Elizabeth Sager : Mark Haedicke'),
 ('recipropcal', 'Elizabeth Sager : Mark Taylor'),
 ('recipropcal', 'Elizabeth Sager : Richard Sanders'),
 ('recipropcal', 'Eric Bass : Susan Scott'),
 ('recipropcal', 'Fletcher Sturm : Greg Whalley'),
 ('recipropcal', 'Fletcher Sturm : Sally Beck'),
 ('recipropcal', 'Gerald Nemec : Susan Scott'),
 ('recipropcal', 'Grant Masson : Vince Kaminski'),
 ('recipropcal', 'Greg Whalley : Richard Sanders'),
 ('recipropcal', 'Janette Elbertson : Mark Taylor'),
 ('recipropcal', 'Janette El